In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
df = pd.read_csv('dataset_formateado.csv')
df = df[df['Sale or Return'] == 'sale']


In [2]:
#Elegir producto del que quiere predecir el stock
producto = 'Shanghaiqing'
#Elegir el tiempo de stock que quiere predecir
intervalo = 'Semanalmente' #['Mensualmente', 'Anualmente','Diariamente']

In [3]:
#Seleccionar el producto del que se quiere predecir
df = df[df['Item Name'] == producto].reset_index(drop=True)
df.head()

,Unnamed: 0,Date,Time,Item Code,Quantity Sold (kilo),Unit Selling Price (EUR/kg),Sale or Return,Discount (Yes/No),Item Name,Category Code,Category Name
0,3,2020-07-01,09:19:45,2,0.421,1.3,sale,No,Shanghaiqing,1011010101,Flower/Leaf Vegetables
1,116,2020-07-01,10:20:38,2,0.241,1.3,sale,No,Shanghaiqing,1011010101,Flower/Leaf Vegetables
2,136,2020-07-01,10:27:59,2,0.661,1.3,sale,No,Shanghaiqing,1011010101,Flower/Leaf Vegetables
3,144,2020-07-01,10:32:52,2,0.491,1.3,sale,No,Shanghaiqing,1011010101,Flower/Leaf Vegetables
4,158,2020-07-01,10:39:54,2,0.210,1.3,sale,No,Shanghaiqing,1011010101,Flower/Leaf Vegetables


In [5]:
# 📌 2. Procesar fecha y Fragmentar fecha y hora 
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'])
if intervalo == 'Anualmente':
    df['Date'] = df['Date'].dt.strftime('%Y')
elif intervalo == 'Mensualmente':
    df['Date'] = df['Date'].dt.strftime('%Y-%m')

df = df.resample('W', on='Date').agg({
        "Quantity Sold (kilo)": "sum",  # Sumamos la cantidad vendida semanalmente
        "Unit Selling Price (EUR/kg)": "mean",  # Promedio del precio unitario
    }).reset_index()
# df.set_index("Date",inplace=True)
# df.drop(columns=['Unnamed: 0'])
# df_grouped = df.groupby(["Date","Item Name"]).agg(
#     {"Quantity Sold (kilo)": "sum", "Unit Selling Price (EUR/kg)": "mean"}
# )
# df_grouped.head()

In [8]:
df.head()

,Date,Quantity Sold (kilo),Unit Selling Price (EUR/kg)
0,2020-07-05,71.626,1.296566
1,2020-07-12,95.119,1.296830
2,2020-07-19,114.242,1.285053
3,2020-07-26,105.463,1.096850
4,2020-08-02,122.346,1.129212


In [6]:
# 📌 4. Selección de variables
features = ["Unit Selling Price (EUR/kg)"]

X = df_grouped[features]
y = df_grouped["Quantity Sold (kilo)"]

# # 📌 5. División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.13, random_state=42)

# # 📌 6. Entrenamiento del modelo
modelo = XGBRegressor(n_estimators=50, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8)
modelo.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=50, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [7]:
# 📌 7. Evaluación
preds = modelo.predict(X_test)
mae = mean_absolute_error(y_test, preds)
print(f"Error MAE: {mae}")

# 📌 8. Predicción para una fecha futura

# fecha_prediccion = 
# nueva_fecha = pd.DataFrame([[2023,7,1, 1.5]], columns=X.columns)
# prediccion = modelo.predict(nueva_fecha)
# print(f"Predicción de ventas: {prediccion[0]} kilos")

#Las predicciones no tienen mucho sentido, cuanto más caro el producto, más se vende.

Error MAE: 4.801384805409113


In [8]:
ultima_fila = X.iloc[-1:].copy()
prediccion_futura = modelo.predict(ultima_fila)[0]

In [9]:
prediccion_futura

8.125163